In [ ]:
#In this notebook I will examine the customer data of a telecommunications company and create a 
#logistic regression model to predict which customers are likely to continue with their subscription.

#To do list
#Create a Dictionary for the encoded categorical variables

In [1]:
#First I will import the necessary packages
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt

In [2]:
#importing the datasets
churn= pd.read_csv ("E:\Data\churn_data.csv")
customer=pd.read_csv ("E:\Data\customer_data.csv")
internet=pd.read_csv ("E:\Data\internet_data.csv") 

print(churn)
print(customer)
print(internet)

      customerID  tenure PhoneService        Contract PaperlessBilling  \
0     7590-VHVEG       1           No  Month-to-month              Yes   
1     5575-GNVDE      34          Yes        One year               No   
2     3668-QPYBK       2          Yes  Month-to-month              Yes   
3     7795-CFOCW      45           No        One year               No   
4     9237-HQITU       2          Yes  Month-to-month              Yes   
...          ...     ...          ...             ...              ...   
7037  2569-WGERO      72          Yes        Two year              Yes   
7038  6840-RESVB      24          Yes        One year              Yes   
7039  2234-XADUH      72          Yes        One year              Yes   
7040  4801-JZAZL      11           No  Month-to-month              Yes   
7041  8361-LTMKD       4          Yes  Month-to-month              Yes   

                  PaymentMethod  MonthlyCharges TotalCharges Churn  
0              Electronic check           

In [3]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7042 entries, 0 to 7041
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7042 non-null   object 
 1   tenure            7042 non-null   int64  
 2   PhoneService      7042 non-null   object 
 3   Contract          7042 non-null   object 
 4   PaperlessBilling  7042 non-null   object 
 5   PaymentMethod     7042 non-null   object 
 6   MonthlyCharges    7042 non-null   float64
 7   TotalCharges      7042 non-null   object 
 8   Churn             7042 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 495.3+ KB


In [4]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7042 entries, 0 to 7041
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7042 non-null   object
 1   gender         7042 non-null   object
 2   SeniorCitizen  7042 non-null   int64 
 3   Partner        7042 non-null   object
 4   Dependents     7042 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB


In [5]:
internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7042 entries, 0 to 7041
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        7042 non-null   object
 1   MultipleLines     7042 non-null   object
 2   InternetService   7042 non-null   object
 3   OnlineSecurity    7042 non-null   object
 4   OnlineBackup      7042 non-null   object
 5   DeviceProtection  7042 non-null   object
 6   TechSupport       7042 non-null   object
 7   StreamingTV       7042 non-null   object
 8   StreamingMovies   7042 non-null   object
dtypes: object(9)
memory usage: 495.3+ KB


In [6]:
#I was running into an issue where TotalCharges column was recognized as an object even though
#it contained numerical value so this is to fix that problem
churn['TotalCharges'] = pd.to_numeric(churn['TotalCharges'], errors = 'coerce')
churn.dtypes

customerID           object
tenure                int64
PhoneService         object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [7]:
#Transposing the data will help me line up the customerID row of each df
chtrans =churn.transpose()
cutrans =customer.transpose()
intrans =internet.transpose()

In [8]:
#Combining data sets
#Thes data frames are organizd by customerID so i can concatenate them then eliminate the duplicate rows after.
ccitrans= pd.concat([chtrans, cutrans, intrans], join="inner")
ccitrans.head(23)
 

,0,1,2,3,4,5,6,7,8,9,...,7032,7033,7034,7035,7036,7037,7038,7039,7040,7041
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU,9305-CDSKC,1452-KIOVK,6713-OKOMC,7892-POOKP,6388-TABGU,...,6894-LFHLY,9767-FFLEM,0639-TSIQW,8456-QDAVC,7750-EYXWZ,2569-WGERO,6840-RESVB,2234-XADUH,4801-JZAZL,8361-LTMKD
tenure,1,34,2,45,2,8,22,10,28,62,...,1,38,67,19,12,72,24,72,11,4
PhoneService,No,Yes,Yes,No,Yes,Yes,Yes,No,Yes,Yes,...,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,No,Yes
Contract,Month-to-month,One year,Month-to-month,One year,Month-to-month,Month-to-month,Month-to-month,Month-to-month,Month-to-month,One year,...,Month-to-month,Month-to-month,Month-to-month,Month-to-month,One year,Two year,One year,One year,Month-to-month,Month-to-month
PaperlessBilling,Yes,No,Yes,No,Yes,Yes,Yes,No,Yes,No,...,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes
PaymentMethod,Electronic check,Mailed check,Mailed check,Bank transfer (automatic),Electronic check,Electronic check,Credit card (automatic),Mailed check,Electronic check,Bank transfer (automatic),...,Electronic check,Credit card (automatic),Credit card (automatic),Bank transfer (automatic),Electronic check,Bank transfer (automatic),Mailed check,Credit card (automatic),Electronic check,Mailed check
MonthlyCharges,29.85,56.95,53.85,42.3,70.7,99.65,89.1,29.75,104.8,56.15,...,75.75,69.5,102.95,78.7,60.65,21.15,84.8,103.2,29.6,74.4
TotalCharges,29.85,1889.5,108.15,1840.75,151.65,820.5,1949.4,301.9,3046.05,3487.95,...,75.75,2625.25,6886.25,1495.1,743.3,1419.4,1990.5,7362.9,346.45,306.6
Churn,No,No,Yes,No,Yes,Yes,No,No,Yes,No,...,Yes,No,Yes,No,No,No,No,No,No,Yes
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU,9305-CDSKC,1452-KIOVK,6713-OKOMC,7892-POOKP,6388-TABGU,...,6894-LFHLY,9767-FFLEM,0639-TSIQW,8456-QDAVC,7750-EYXWZ,2569-WGERO,6840-RESVB,2234-XADUH,4801-JZAZL,8361-LTMKD


In [9]:
#Here I drop the extra customerID rows then transpose the data set to get it back to its original orientation.
#cci is an abreviation for each original data frame
cci = ccitrans.drop_duplicates().transpose()


In [10]:
 cci.dtypes

customerID          object
tenure              object
PhoneService        object
Contract            object
PaperlessBilling    object
PaymentMethod       object
MonthlyCharges      object
TotalCharges        object
Churn               object
gender              object
SeniorCitizen       object
Partner             object
Dependents          object
MultipleLines       object
InternetService     object
OnlineSecurity      object
OnlineBackup        object
DeviceProtection    object
TechSupport         object
StreamingTV         object
StreamingMovies     object
dtype: object

In [11]:
#A majority of this data sets variables are cetegorical but there are 3 rows with numerical values.
#Here I change the data types of those rows into integers and floats 
cci = cci.astype({'TotalCharges': float})
cci = cci.astype({'MonthlyCharges': float})
cci = cci.astype({'tenure': int},)
cci.dtypes

customerID           object
tenure                int32
PhoneService         object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
gender               object
SeniorCitizen        object
Partner              object
Dependents           object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
dtype: object

In [12]:
#Now we will examine the missing data
#False means data is not missing
import numpy as np

# replace "?" to NaN
cci.replace("?", np.nan, inplace = True)
cci.head(5)
missing = cci.isnull()
missing.head(5)

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,gender,...,Partner,Dependents,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
#Using a for loop I can return the number of missing values in each column
for column in missing.columns.values.tolist():
    print(column)
    print (missing[column].value_counts())
#Since the amount of False values are listed at 7042 we can see that this data set is not missing values.

customerID
False    7042
Name: customerID, dtype: int64
tenure
False    7042
Name: tenure, dtype: int64
PhoneService
False    7042
Name: PhoneService, dtype: int64
Contract
False    7042
Name: Contract, dtype: int64
PaperlessBilling
False    7042
Name: PaperlessBilling, dtype: int64
PaymentMethod
False    7042
Name: PaymentMethod, dtype: int64
MonthlyCharges
False    7042
Name: MonthlyCharges, dtype: int64
TotalCharges
False    7031
True       11
Name: TotalCharges, dtype: int64
Churn
False    7042
Name: Churn, dtype: int64
gender
False    7042
Name: gender, dtype: int64
SeniorCitizen
False    7042
Name: SeniorCitizen, dtype: int64
Partner
False    7042
Name: Partner, dtype: int64
Dependents
False    7042
Name: Dependents, dtype: int64
MultipleLines
False    7042
Name: MultipleLines, dtype: int64
InternetService
False    7042
Name: InternetService, dtype: int64
OnlineSecurity
False    7042
Name: OnlineSecurity, dtype: int64
OnlineBackup
False    7042
Name: OnlineBackup, dtype: int64
De

In [14]:
#This data set contains many catergorical values that must be encoded to use with my model
#Here I create a new df containing only object data from cci
obj_df = cci.select_dtypes(include=['object']).copy() 

In [15]:
#Here we encode our categorical data.
for column in obj_df:
    #This changes the data types from object to category
    obj_df[column] = obj_df[column].astype('category')
    
    
    #This creates new columns with our categorical data encoded
    obj_df[column+'code'] = obj_df[column].cat.codes
    


In [16]:
obj_df

,customerID,PhoneService,Contract,PaperlessBilling,PaymentMethod,Churn,gender,Partner,Dependents,MultipleLines,...,Partnercode,Dependentscode,MultipleLinescode,InternetServicecode,OnlineSecuritycode,OnlineBackupcode,DeviceProtectioncode,TechSupportcode,StreamingTVcode,StreamingMoviescode
0,7590-VHVEG,No,Month-to-month,Yes,Electronic check,No,Female,Yes,No,No phone service,...,1,0,1,0,0,2,0,0,0,0
1,5575-GNVDE,Yes,One year,No,Mailed check,No,Male,No,No,No,...,0,0,0,0,2,0,2,0,0,0
2,3668-QPYBK,Yes,Month-to-month,Yes,Mailed check,Yes,Male,No,No,No,...,0,0,0,0,2,2,0,0,0,0
3,7795-CFOCW,No,One year,No,Bank transfer (automatic),No,Male,No,No,No phone service,...,0,0,1,0,2,0,2,2,0,0
4,9237-HQITU,Yes,Month-to-month,Yes,Electronic check,Yes,Female,No,No,No,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7037,2569-WGERO,Yes,Two year,Yes,Bank transfer (automatic),No,Female,No,No,No,...,0,0,0,2,1,1,1,1,1,1
7038,6840-RESVB,Yes,One year,Yes,Mailed check,No,Male,Yes,Yes,Yes,...,1,1,2,0,2,0,2,2,2,2
7039,2234-XADUH,Yes,One year,Yes,Credit card (automatic),No,Female,Yes,Yes,Yes,...,1,1,2,1,0,2,2,0,2,2
7040,4801-JZAZL,No,Month-to-month,Yes,Electronic check,No,Female,Yes,Yes,No phone service,...,1,1,1,0,2,0,0,0,0,0


In [17]:
#Combining our new df with encoded values
objtrans = obj_df.transpose()
ccitrans = cci.transpose()
cciwcode =  pd.concat([objtrans, ccitrans], join="inner")


In [18]:
#changing orientation of df
cci = cciwcode.drop_duplicates().transpose()

In [19]:
#Turing our dependent variable into 1 and 0  
cci['Churn'+'code'] = cci.Churn.replace(to_replace=['No','Yes'], value=[0,1])

In [20]:
cci

,customerID,PhoneService,Contract,PaperlessBilling,PaymentMethod,Churn,gender,Partner,Dependents,MultipleLines,...,OnlineSecuritycode,OnlineBackupcode,DeviceProtectioncode,TechSupportcode,StreamingTVcode,StreamingMoviescode,tenure,MonthlyCharges,TotalCharges,SeniorCitizen
0,7590-VHVEG,No,Month-to-month,Yes,Electronic check,No,Female,Yes,No,No phone service,...,0,2,0,0,0,0,1,29.85,29.85,0
1,5575-GNVDE,Yes,One year,No,Mailed check,No,Male,No,No,No,...,2,0,2,0,0,0,34,56.95,1889.5,0
2,3668-QPYBK,Yes,Month-to-month,Yes,Mailed check,Yes,Male,No,No,No,...,2,2,0,0,0,0,2,53.85,108.15,0
3,7795-CFOCW,No,One year,No,Bank transfer (automatic),No,Male,No,No,No phone service,...,2,0,2,2,0,0,45,42.3,1840.75,0
4,9237-HQITU,Yes,Month-to-month,Yes,Electronic check,Yes,Female,No,No,No,...,0,0,0,0,0,0,2,70.7,151.65,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7037,2569-WGERO,Yes,Two year,Yes,Bank transfer (automatic),No,Female,No,No,No,...,1,1,1,1,1,1,72,21.15,1419.4,0
7038,6840-RESVB,Yes,One year,Yes,Mailed check,No,Male,Yes,Yes,Yes,...,2,0,2,2,2,2,24,84.8,1990.5,0
7039,2234-XADUH,Yes,One year,Yes,Credit card (automatic),No,Female,Yes,Yes,Yes,...,0,2,2,0,2,2,72,103.2,7362.9,0
7040,4801-JZAZL,No,Month-to-month,Yes,Electronic check,No,Female,Yes,Yes,No phone service,...,2,0,0,0,0,0,11,29.6,346.45,0


In [22]:
#Creating a dataframe containing the variables for my model
model = cci[['Churncode','tenure','PaymentMethodcode','TechSupportcode','Contractcode','InternetServicecode','Dependentscode']]

In [23]:
Pay_key = obj_df[['PaymentMethod','PaymentMethodcode']].drop_duplicates()
Pay_key

,PaymentMethod,PaymentMethodcode
0,Electronic check,2
1,Mailed check,3
3,Bank transfer (automatic),0
6,Credit card (automatic),1


In [24]:
#Below Is a key for each categoircal variable.
Tech_key = obj_df[['TechSupport','TechSupportcode']].drop_duplicates()
Tech_key

,TechSupport,TechSupportcode
0,No,0
3,Yes,2
11,No internet service,1


In [25]:
Cont_key = obj_df[['Contract','Contractcode']].drop_duplicates()
Cont_key

,Contract,Contractcode
0,Month-to-month,0
1,One year,1
11,Two year,2


In [26]:
Int_key = obj_df[['InternetService','InternetServicecode']].drop_duplicates()
Int_key

,InternetService,InternetServicecode
0,DSL,0
4,Fiber optic,1
11,No,2


In [27]:
Dep_key = obj_df[['Dependents','Dependentscode']].drop_duplicates()
Dep_key

,Dependents,Dependentscode
0,No,0
6,Yes,1


In [ ]:
#Defining X and y of the data set
X = np.asarray(model.drop('Churncode', axis = 'columns'))
y = np.asarray(model['Churncode'])


In [ ]:
#Normalizing the dataset
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

In [ ]:
#Splitting the dataset into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
LR

In [ ]:
#Making Predictions using the test I'vs created
yhat = LR.predict(X_test)
yhat

In [ ]:
#Probability of a customer resubscribing(Column 1) or leaving (column)
yhat_prob = LR.predict_proba(X_test)
yhat_prob

In [ ]:
#Evaluating our model
from sklearn.metrics import jaccard_score
jaccard_score(y_test, yhat)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
cci.corr()